In [2]:
# Setup.
import pandas as pd
import requests
import json
import matplotlib as plt
import numpy as np
import itertools 
import warnings
warnings.filterwarnings("ignore")

# Define a function that will loop through 'Results' of each API call
# and generate a dictionary per set of Results.
def get_qualifying_race_results(year_number,round_number):
    url = 'http://ergast.com/api/f1/{}/{}/results.json'.format(year_number, round_number)
    try:
        resp = requests.get(url).json()#.get('MRData').get('RaceTable').get('Races')[0].get('Results')
        results = resp.get('MRData').get('RaceTable').get('Races')[0].get('Results')
        circuit = resp.get('MRData').get('RaceTable').get('Races')[0].get('Circuit')
        temp_df =  pd.DataFrame.from_records(results)
        temp_df['circuitName'] = circuit.get('circuitName')
        temp_df['lat'] = circuit.get('Location').get('lat')
        temp_df['long'] = circuit.get('Location').get('long')
        temp_df['country'] = circuit.get('Location').get('country')
        temp_df['locality'] = circuit.get('Location').get('locality')
        temp_df['round'] = round_number
        temp_df['year'] = year_number
        return temp_df
    # Adding a pass here as not all years have the same number of rounds.
    # In order for it to complete when there is no round for that year,
    # we need to "pass" that row.
    except:
        pass

# Create list of years and rounds to loop through when pooling data.

seasons = np.arange(2003,2018,1)
rounds = np.arange(1,20,1)

# Create a list of arrays which is required for itertools to create the combination of the two lists.
seasons_rounds = [seasons,rounds]

# Create a list of pairs of every combination of years and round.
seasons_rounds = list(itertools.product(*seasons_rounds))

# Initiate empty list within which we will store the dataframes outputted by the function get_qualifying_races.
results = []

# Unpack the year/round tuples and run each set through the API call function above.
# Store the results in the empty list 'results'.
for combo in seasons_rounds:
    yr, r = combo
    temp_result =  get_qualifying_race_results(yr, r)
    results.append(temp_result)
    
df = pd.concat(results, axis=0)

# Extract driverId into its own column.
df['driverId']= df.Driver.map(lambda x: x.get('driverId'))

df.drop(['Time','Constructor','FastestLap'], axis=1, inplace=True)
df.head()

,Driver,circuitName,country,grid,laps,lat,locality,long,number,points,position,positionText,round,status,year,driverId
0,"{'driverId': 'coulthard', 'code': 'COU', 'url'...",Albert Park Grand Prix Circuit,Australia,11,58,-37.8497,Melbourne,144.968,5,10,1,1,1,Finished,2003,coulthard
1,"{'driverId': 'montoya', 'code': 'MON', 'url': ...",Albert Park Grand Prix Circuit,Australia,3,58,-37.8497,Melbourne,144.968,3,8,2,2,1,Finished,2003,montoya
2,"{'driverId': 'raikkonen', 'permanentNumber': '...",Albert Park Grand Prix Circuit,Australia,15,58,-37.8497,Melbourne,144.968,6,6,3,3,1,Finished,2003,raikkonen
3,"{'driverId': 'michael_schumacher', 'code': 'MS...",Albert Park Grand Prix Circuit,Australia,1,58,-37.8497,Melbourne,144.968,1,5,4,4,1,Finished,2003,michael_schumacher
4,"{'driverId': 'trulli', 'code': 'TRU', 'url': '...",Albert Park Grand Prix Circuit,Australia,12,58,-37.8497,Melbourne,144.968,7,4,5,5,1,Finished,2003,trulli
